### Featurize IrOx slabs from Seoin
---

In [1]:
# #########################################################
# This haven't been done
# #########################################################
# ('features', 'o', 'O_magmom'),
# ('features', 'o', 'Ir_magmom'),
# ('features', 'o', 'Ir*O_bader'),
# ('features', 'o', 'Ir_bader'),
# ('features', 'o', 'O_bader'),
# ('features', 'o', 'p_band_center'),



# #########################################################
# These are done
# #########################################################
# ('features', 'o', 'bulk_oxid_state'),
# ('features', 'o', 'angle_O_Ir_surf_norm'),
# ('features', 'o', 'active_o_metal_dist'),
# ('features', 'o', 'effective_ox_state'),
# ('features', 'o', 'ir_o_mean'),
# ('features', 'o', 'ir_o_std'),
# ('features', 'o', 'octa_vol'),
# ('features', 'o', 'dH_bulk'),
# ('features', 'o', 'volume_pa'),

### Import Modules

In [2]:
import os
print(os.getcwd())
import sys
import time; ti = time.time()

import pickle
from pathlib import Path

import pandas as pd
import numpy as np

# #########################################################
from methods_features import get_octa_geom, get_octa_vol
from methods_features import get_angle_between_surf_normal_and_O_Ir

# #########################################################
from local_methods import get_df_coord_local
from local_methods import get_effective_ox_state

from methods_features import get_octahedra_atoms
from methods_features import get_octahedral_oxygens_A
from methods import get_metal_active_site

from proj_data import metal_atom_symbol

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/seoin_irox_data/featurize_data


In [3]:
from methods_features import get_more_octahedra_data

In [4]:
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.options.display.max_colwidth = 100

In [5]:
dir_i = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/seoin_irox_data")

# #########################################################
path_i = os.path.join(
    dir_i, "out_data/df_ads_e.pickle")
with open(path_i, "rb") as fle:
    df_ads_e = pickle.load(fle)

# #########################################################
path_i = os.path.join(
    dir_i, "out_data/df_oer.pickle")
with open(path_i, "rb") as fle:
    df_oer = pickle.load(fle)

# #########################################################
path_i = os.path.join(
    dir_i, "process_bulk_data",
    "out_data/df_seoin_bulk.pickle")
with open(path_i, "rb") as fle:
    df_bulk = pickle.load(fle)
df_bulk = df_bulk.set_index("crystal")

In [6]:
# # TEMP
# print(111 * "TEMP | ")

# df_ads_e = df_ads_e.dropna(axis=0, subset=["active_site__o", "active_site__oh", "active_site__ooh"])

In [7]:
df_ads_e.shape

(174, 17)

In [8]:
# # TEMP
# print(111 * "TEMP | ")
# df_ads_e = df_ads_e.sample(n=10)

In [9]:
data_dict_list = []
for name_i, row_i in df_ads_e.iterrows():
    # print(name_i)


    # #####################################################
    name_dict_i = dict(zip(
        df_ads_e.index.names,
        name_i))
    # #####################################################
    name_str_i = row_i["name"]
    index_o_i = row_i.index_o
    active_site_o_i = row_i.active_site__o
    # bulk_oxid_state_i = row_i.bulk_oxid_state
    # #####################################################
    crystal_i = name_dict_i["crystal"]
    # #####################################################

    # #####################################################
    row_oer_o_i = df_oer.loc[index_o_i]
    # #####################################################
    atoms_o_i = row_oer_o_i.atoms
    atoms_o_init_i = row_oer_o_i.atoms_init
    # #####################################################

    # #####################################################
    row_bulk_i = df_bulk.loc[crystal_i]
    # #####################################################
    volume_pa_i = row_bulk_i.volume_pa
    dH_i = row_bulk_i.dH
    # #####################################################



    df_coord_o_final_i = get_df_coord_local(
        name=name_str_i,
        ads="o",
        atoms=atoms_o_i,
        append_str="_final",
        )
    df_coord_o_init_i = get_df_coord_local(
        name=name_str_i,
        ads="o",
        atoms=atoms_o_init_i,
        append_str="_init",
        )

    eff_ox_out_i = get_effective_ox_state(
        active_site=active_site_o_i,
        df_coord_i=df_coord_o_final_i,
        df_coord_init_i=df_coord_o_init_i,
        metal_atom_symbol="Ir",
        )
    eff_ox_i = eff_ox_out_i["effective_ox_state"]


    # #####################################################
    # Octahedral geometry
    octa_geom_out = get_octa_geom(
        df_coord_i=df_coord_o_final_i,
        active_site_j=active_site_o_i,
        atoms=atoms_o_i,
        verbose=True,
        )
    for key_i in octa_geom_out.keys():
        octa_geom_out[key_i + "__o"] = octa_geom_out.pop(key_i)



    # #####################################################
    # Ir-O Angle relative to surface normal
    angle_i = get_angle_between_surf_normal_and_O_Ir(
        atoms_o_i,
        df_coord=df_coord_o_final_i,
        active_site=active_site_o_i,
        )























    metal_active_site_i = get_metal_active_site(
        df_coord=df_coord_o_final_i,
        active_site=active_site_o_i,
        metal_atom_symbol=metal_atom_symbol,
        )

    octahedral_oxygens, octahedral_oxygens_images = get_octahedral_oxygens_A(
        df_coord=df_coord_o_final_i,
        metal_active_site=metal_active_site_i,
        )

    data_out = get_octahedra_atoms(
        octahedral_oxygens=octahedral_oxygens,
        df_coord=df_coord_o_final_i,
        atoms=atoms_o_i,
        active_site=active_site_o_i,
        metal_active_site=metal_active_site_i,
        ads="o",
        )
    octahedral_oxygens = data_out["octahedral_oxygens"]
    missing_active_site = data_out["missing_active_site"]


    from methods import get_oxy_images
    oxy_images_i = get_oxy_images(
        atoms=atoms_o_i,
        octahedral_oxygens=octahedral_oxygens,
        metal_active_site=metal_active_site_i,
        )


    process_further = True
    if missing_active_site is None:
        process_further = False

    data_out_2 = dict()
    if process_further :
        data_out_2 = get_more_octahedra_data(
            atoms=atoms_o_i,
            oxy_images=oxy_images_i,
            active_site=active_site_o_i,
            metal_active_site=metal_active_site_i,
            octahedral_oxygens=octahedral_oxygens,
            )









    verbose = True
    octa_vol_i = get_octa_vol(
        df_coord_i=df_coord_o_final_i,
        active_site_j=active_site_o_i,

        octahedra_atoms=octahedral_oxygens,
        metal_active_site=metal_active_site_i,

        atoms=atoms_o_i,
        verbose=verbose,
        )







    # #####################################################
    data_dict_i = dict()
    # #####################################################
    data_dict_i["effective_ox_state__o"] = eff_ox_i
    data_dict_i["octa_vol__o"] = octa_vol_i
    data_dict_i["angle_O_Ir_surf_norm__o"] = angle_i
    data_dict_i["dH_bulk"] = dH_i
    data_dict_i["volume_pa"] = volume_pa_i
    # data_dict_i["bulk_oxid_state"] = bulk_oxid_state_i
    # #####################################################
    data_dict_i.update(octa_geom_out)
    data_dict_i.update(name_dict_i)
    data_dict_i.update(data_out_2)
    # #####################################################
    data_dict_list.append(data_dict_i)
    # #####################################################

    # print("")

# #########################################################
df_feat = pd.DataFrame(data_dict_list)
df_feat = df_feat.set_index(df_ads_e.index.names)

df_features_targets = pd.concat([
    df_feat,
    df_ads_e.drop(columns=["O_Ir_frac_ave", ])
    ], axis=1)
# #########################################################

This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?


/home/raul_desktop/anaconda3/envs/PROJ_irox_oer/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/raul_desktop/anaconda3/envs/PROJ_irox_oer/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

/home/raul_desktop/anaconda3/envs/PROJ_irox_oer/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/raul_desktop/anaconda3/envs/PROJ_irox_oer/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning:

invalid value encountered in true_divide

/home/raul_desktop/anaconda3/envs/PROJ_irox_oer/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars



This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basically always an issue right?
This is basicall

In [10]:
# for i in df_features_targets.columns.to_list():
#     print(i)

### Processing columns

In [11]:
df_features_targets.columns.tolist()


multicolumn_assignments = {

    # #######################
    # Features ##############
    "effective_ox_state__o":   ("features", "effective_ox_state", "", ),

    # "effective_ox_state__o":   ("features", "o", "effective_ox_state", ),
    "octa_vol__o":             ("features", "o", "octa_vol", ),
    "active_o_metal_dist__o":  ("features", "o", "active_o_metal_dist", ),
    "ir_o_mean__o":            ("features", "o", "ir_o_mean", ),
    "ir_o_std__o":             ("features", "o", "ir_o_std", ),
    "angle_O_Ir_surf_norm__o": ("features", "o", "angle_O_Ir_surf_norm", ),

    "bulk_oxid_state":         ("features", "bulk_oxid_state", "", ),
    "dH_bulk":                 ("features", "dH_bulk", "", ),
    "volume_pa":               ("features", "volume_pa", "", ),

    "oxy_opp_as_bl":                   ("features", "o", "oxy_opp_as_bl", ),
    "degrees_off_of_straight__as_opp": ("features", "o", "degrees_off_of_straight__as_opp", ),


    # #######################
    # Targets ###############
    "e_o":   ("targets", "e_o", "", ),
    "e_oh":  ("targets", "e_oh", "", ),
    "e_ooh": ("targets", "e_ooh", "", ),
    "g_o":   ("targets", "g_o", "", ),
    "g_oh":  ("targets", "g_oh", "", ),
    "g_ooh": ("targets", "g_ooh", "", ),


    # #######################
    # Data ##################
    "index_bare":          ("data", "index_bare", "", ),
    "index_o":             ("data", "index_o", "", ),
    "index_oh":            ("data", "index_oh", "", ),
    "index_ooh":           ("data", "index_ooh", "", ),
    "name":                ("data", "name", "", ),
    "active_site__o":      ("data", "active_site__o", "", ),
    "active_site__oh":     ("data", "active_site__oh", "", ),
    "active_site__ooh":    ("data", "active_site__ooh", "", ),
    "oxy_opposite_as":     ("data", "oxy_opposite_as", "", ),

    "stoich":    ("data", "stoich", "", ),

    }

In [12]:
new_cols = []
for col_i in df_features_targets.columns:
    new_col_i = multicolumn_assignments.get(col_i, col_i)
    if col_i not in multicolumn_assignments.keys():
        print("Oh no, this column needs to be added multicolumn_assignments")
        print(col_i)

    new_cols.append(new_col_i)

idx = pd.MultiIndex.from_tuples(new_cols)
df_features_targets.columns = idx

In [13]:
df_features_targets = df_features_targets.reindex(columns=[
    "targets",
    "data",
    "format",
    "features",
    "features_pre_dft",
    "features_stan",
    ], level=0)

In [14]:
df_features_targets = df_features_targets.sort_index(axis=1)

In [15]:
other_cols = []
other_feature_cols = []
ads_feature_cols = []
for col_i in df_features_targets.columns:
    if col_i[0] == "features":
        if col_i[1] in ["o", "oh", "ooh", ]:
            ads_feature_cols.append(col_i)
        else:
            other_feature_cols.append(col_i)
    else:
        other_cols.append(col_i)

In [16]:
df_features_targets = df_features_targets[
    other_cols + other_feature_cols + ads_feature_cols
    ]

In [17]:
df_features_targets

data  \
                                                 active_site__o   
                                                                  
crystal facet coverage   termination active_site                  
amm2    012   OH_covered 0           0                       50   
        100   O_covered  0           0                       32   
        110   O_covered  0           0                       48   
                                     1                       37   
                                     2                       49   
...                                                         ...   
rutile  120   O_covered  1           0                       37   
                                     1                       38   
        121   O_covered  0           0                       48   
                                     1                       49   
                                     2                       50   

                                                                  \
                                                 active_site__oh   
                                                                   
crystal facet coverage   termination active_site                   
amm2    012   OH_covered 0           0                        48   
        100   O_covered  0           0                        32   
        110   O_covered  0           0                        48   
                                     1                        37   
                                     2                        49   
...                                                          ...   
rutile  120   O_covered  1           0                        37   
                                     1                        38   
        121   O_covered  0           0                        48   
                                     1                        49   
                                     2                        50   

                                                                              \
                                                 active_site__ooh index_bare   
                                                                               
crystal facet coverage   termination active_site                               
amm2    012   OH_covered 0           0                         48       1078   
        100   O_covered  0           0                         32       1070   
        110   O_covered  0           0                         48       1030   
                                     1                         37       1031   
                                     2                         49       1032   
...                                                           ...        ...   
rutile  120   O_covered  1           0                         37         93   
                                     1                         38         94   
        121   O_covered  0           0                         48          0   
                                     1                         49          1   
                                     2                         50          2   

                                                                             \
                                                 index_o index_oh index_ooh   
                                                                              
crystal facet coverage   termination active_site                              
amm2    012   OH_covered 0           0              1085     1086      1089   
        100   O_covered  0           0              1071     1073      1077   
        110   O_covered  0           0              1033     1036      1044   
                                     1              1033     1037      1046   
                                     2              1033     1040      1047   
...                                                  ...      ...       ...   
rutile  120   O_covered  1           0   

### Write data to file

In [18]:
# Pickling data ###########################################
directory = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/seoin_irox_data/featurize_data",
    "out_data")
if not os.path.exists(directory):
    os.makedirs(directory)

path_i = os.path.join(directory, "df_features_targets.pickle")
with open(path_i, "wb") as fle:
    pickle.dump(df_features_targets, fle)
# #########################################################

In [19]:
print("df_features_targets.shape:", df_features_targets.shape)

df_features_targets.head()

df_features_targets.shape: (174, 27)


data  \
                                                 active_site__o   
                                                                  
crystal facet coverage   termination active_site                  
amm2    012   OH_covered 0           0                       50   
        100   O_covered  0           0                       32   
        110   O_covered  0           0                       48   
                                     1                       37   
                                     2                       49   

                                                                  \
                                                 active_site__oh   
                                                                   
crystal facet coverage   termination active_site                   
amm2    012   OH_covered 0           0                        48   
        100   O_covered  0           0                        32   
        110   O_covered  0           0                        48   
                                     1                        37   
                                     2                        49   

                                                                              \
                                                 active_site__ooh index_bare   
                                                                               
crystal facet coverage   termination active_site                               
amm2    012   OH_covered 0           0                         48       1078   
        100   O_covered  0           0                         32       1070   
        110   O_covered  0           0                         48       1030   
                                     1                         37       1031   
                                     2                         49       1032   

                                                                             \
                                                 index_o index_oh index_ooh   
                                                                              
crystal facet coverage   termination active_site                              
amm2    012   OH_covered 0           0              1085     1086      1089   
        100   O_covered  0           0              1071     1073      1077   
        110   O_covered  0           0              1033     1036      1044   
                                     1              1033     1037      1046   
                                     2              1033     1040      1047   

                                                                           \
                                                                     name   
                                                                            
crystal facet coverage   termination active_site                            
amm2    012   OH_covered 0           0            amm2_012_OH_covered_0_0   
        100   O_covered  0           0             amm2_100_O_covered_0_0   
        110   O_covered  0           0             amm2_110_O_covered_0_0   
                                     1             amm2_110_O_covered_0_1   
                                     2             amm2_110_O_covered_0_2   

                                                                         \
                                                 oxy_opposite_as stoich   
                                                                          
crystal facet coverage   termination active_site                          
amm2    012   OH_covered 0           0                        35    AB3   
        100   O_covered  0           0                        20    AB3   
        110   O_covered  0           0                        36    AB3   
                                     1                        21    AB3   
                                     2                        45    AB3   

                                       

In [20]:
# #########################################################
print(20 * "# # ")
print("All done!")
print("Run time:", np.round((time.time() - ti) / 60, 3), "min")
print("featurize_data.ipynb")
print(20 * "# # ")
# #########################################################

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
All done!
Run time: 0.832 min
featurize_data.ipynb
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
